# **Cyclic Voltametry**
Aubrey Stevens

In [1]:
import asyncio
import re
import sys
import time
import math

import numpy as np
import pandas as pd

import scipy as sp
import scipy.signal


import serial
import serial.tools.list_ports


import bokeh.plotting
import bokeh.io
import bokeh.layouts
import bokeh.driving
bokeh.io.output_notebook()

notebook_url = "localhost:8888"

Loading BokehJS ...

# **Section 1: Overview:**
In the design of my cyclic voltammeter, I had three main objectives. I wanted to build a circuit that produced clean, consistent data, be able to accurately compute concentrations, and create a user-friendly interface. To complete a circuit that produced a clear signal, the electrode needed to receive a clear triangle wave of varying voltages and the signal from the working electrode would also need to be tuned. Having a clear, accurate signal is particularly important for being able to accurately compute concentrations, as this requires determining the peak anodal current from the duck plots. To accurately measure concentration, I would need to accurately compute D, the diffusion coefficient, by producing a linear regression against either known concentrations or the scan rate. Then, I could solve for the concentration by varying the scan rate and computing another linear regression.

To create a user-friendly interface, I wanted the user to be able to be able to see in real time how the triangle wave and signal from the electrode were being affected by adjusting different parameters in addition to being able to see the duck plot update in real time. This would require streaming in two plots, one of voltage as a function of time for both the triangular wave being input to the electrode and the cleaned up voltage signal from the working electrode, the other of the duck plot. I wanted the user to be able to easily stream in data, be able to reset the data, save a csv of the data, and shut down the kernal with clearly labeled buttons. I also wanted the user to be able to easily adjust the scan rate, minimum and maximum voltages being sent to the electrode, and the number of cycles of the triangle wave would be completed for selected criteria. Sliders seemed to be a nice user-friendly way to adjust these parameters. Overall, I wanted to create a cyclic voltammeter that is both easy to use and is accurate.

# **Section 2: Design and Design Considerations:**

## **Circuit:**
### Communication with DAC:
Since we need to send a triangle wave through the electrode, I figured that communicating with a DAC through I2C would give more control over the sent voltages than using something like PWM. Since we are rapidly changing the voltage to produce a triangle wave, PWM would not be effective, as it needs time to attain the correct average voltage (as it creates the voltage from varying between 0 and 5 V over time. This variation would also produce a quite noisy triangle wave. However, using I2C allows us to more accurately change the voltage to produce the triangle wave. Additionally, using PWM, you can only output values between 0-255 for the duty ratio. However, with I2C, we can use values between 0-4095, giving us much more control over the voltage we send out as well.

Thus, I chose to use an MCP4725 chip to output the triangle wave. I chose this instead of the ADC chip, because while the ADC chip can read in negative voltages, it can't output negative voltages directly, meaning it wouldn't be particularly beneficial over the MCP4725 chip. Thus, I chose to use the simpler chip, as I would need to shift down the triangle wave regardless.

### Unity Gain Amplifier:
My triangle wave had a bit of noise when directly output from the MCP4725. This was a result of there being a finite number of points on the wave and only being able to send out integer values for voltage through the MCP. This resulted in triangle wave that looked a bit like a staircase when zoomed in. Thus, adding a unity gain amplifier would aid in helping with a bit of the noise, as it simply ensures that the output voltage equals the positive input voltage, helping reduce additional noise and smooth out the curve. 

Unity gain amplifiers work as a voltage buffer. Since op amps have a high input impedence and low output impedence, they work to maintain voltages. Since it tries to reduce changes in voltage, this helps it filter out small shifts in voltages (in this case noise from the staircase produced from the triangle wave). Since there wasn't a ton of noise and I still wanted the voltage to change rapidly (and know what the voltages in the triangle were at a given time), I chose not to use a low-pass filter, as this could filter out some of the voltage information I wanted to get to the electrode.

### Difference Amplifier:
I decided to use as much of the range of the DAC as possible. As a result, I needed to compress the signal for a total range of 3 V and also shift it downwards. In the end, for a triangle wave ranging from 0-5V, I wanted it to range from -1.5-1.5 V. A difference amplifier could accomplish this. 

$V_{out} = \frac{R_2}{R_1} (V_2-V_1)$

Since I wanted to be able to vary the minimum and maximum voltages, I decided to make $V_1$ variable using PWM. Since I wanted the voltage range to change from 5 to 3, I needed to select resistors with a ratio of 3:5. Thus, I selected $R_2 = 33 k \Omega$ and $R_1 = 57 k \Omega$.

### Low Pass Filter for PWM:
Since I decided to use PWM for my difference amplifier, I needed to filter out noise caused by PWM before it went to the difference amplifier. Since PWM produces different voltages by varying between 0 and 5 V for different time intervals, this produces a lot of high frequency noise. If left unfiltered, this would create a lot of noise in my triangle wave and would not result in even variation of different voltages over time being fed into the electrode. Thus, I chose to filter the noise from the PWM signal out before it went to my difference amplifier.

To do this, I decided to use a Sallen-Key low-pass filter. I initially chose a cutoff frequency of 100 Hz but the signal was still a bit noisy, so I instead chose a cutoff frequency of 50 Hz, which seemed to drastically improve the signal. I computed the values of capacitors and resistors using the below equations and trying different resistor values against the 0.1 uF capacitors.

$f_{cut} = \frac{1}{2\pi RC}$, where $f_{cut}$ is the cutoff frequency of 50 Hz
\
Solving for R, we get:
$R = \frac{1}{2\pi C f_{cut}}$
$R = \frac{1}{10\pi C}$
$R = \frac{1}{10\pi 0.1 * 10^{-6}}$
$R = 33 k \Omega$

I ended up choosing resistors with $33 k \Omega$ resistance.

### Electrode:
I set up the potentiostat as shown on the cyclic voltameter webpage. This was with the counter electrode (CE) being connected to the output of the op-amp, the reference electrode (RE) being connected to the negative input of the op amp, and the working electrode (WE) being connected to a C-V converter. I chose to tape my electrode connector to a box to restrict its movement to ensure liquid on the elctrode did not spill onto the rest of my electronics.

### Transimpedence Amplifier:
The signal coming out of the working electrode is in the form of a current. Op amps have a very high gain, meaning that essentially all of the current goes through the resistor. Additionally, I chose to add a capacitor to the TIA to prevent ringing. I chose a capacitor value based on what seemed to reduce ringing sufficiently, landing on 0.47 uF. For the resistor, I played around different resistances, seeing what the output voltages looked like. Some of the larger resistances produced too large of a signal, which was out of range of the Arduino. Too small of a resistance resulted in barely being able to see a signal from the working electrode. I landed on $680 \Omega$ as a good resistance.


### Unity Gain Amplifier:
After the transimpedence amplifier, my signal had a bit of noise, so I decided to use another unity gain amplifier to reduce some of this. I chose not to use a low-pass filter, because while there was a bit of noise, it wasn't significant and I didn't want to filter out too much data from the signal.

### Inverting Amplifier:
The signal that comes out of the transimpedence amplifier is inverts the signal from the working electrode. Thus, the signal needs to be inverted again to return it to its original state. I did this by creating an inverting op amp. I selected resistances based on the equation below. I chose to use two $330 \Omega$ resistors for my resistances, as it didn't really matter which values I chose, as long as they were the same, since I didn't want to change the scale of my signal in this amplifier, only invert the data.

### Summing Amplifier:
Since some of the voltages I was going to be reading in would be negative, I decided to create a summing amplifier to ensure I only produced a positive signal. A summing amplifier works by adding together the desired signal and some positive voltage, which are weighted based on different resistances. I began by selecting resistors of the same value (220 $\Omega$) and tested out different resistances until I produced an entirely positive signal that didn't exceed the bounds Arduino could read in (0-5V) and was a clear signal. I landed on using a $330 \Omega$ resistor for the signal from the working electrode and $510$ for the resistor coming from the 3.3V pin of the Arduino.

![](cyclic_volt_bb.jpg)
![](IMG_1009.jpg)

## **Arduino Sketch:**

I created several Arduino Sketches to test out different methods of producing triangle waves and check that parts of my circuit or Arduino code were working properly. Additionally, I wrote both files that interacted with Python and ones that didn't. The purpose of the files that didn't communicate with Python was to test out if my math for producing triangle waves was correct and if it varied properly with variables like the scan rate. The finalized file contains all of the correct information gained from the other sketches with additional code, so I did not include the other files in this Jupyter notebook. However, the scratch files are in the folder labeled `scratch_sketches`. 

The sketch I wrote is loosely based on the Arduino sketch given in lesson 16 of the 189 website. I loosely based my sketch on this, because I wanted to stream data into Python from Arduino and be able to control the Arduino and signals being sent to it through Python. However, I made a lot of adjustments to the given sketch, which I describe below.

### Initializing:
I began the sketch by initializing a lot of variables. Since there are so many, I will use a list format.
- Imported the Adafruit package for the MCP4725 and defined the MCP4725 address.
- HANDSHAKE, VOLTAGE_REQUEST, ON_REQUEST, STREAM, READ_DAQ_DELAY: These are all variables that were originally in the sketch from lesson 16. They are for serial communication with Python and allow for the sketch to be controlled by Python with various commands through callbacks and other functions.
- `SCAN_RATE`, `V_MIN`, `V_MAX`, `N_CYCLES`: These are all variables I created that expand the ability of Python to control various commands in the sketch. They all need to have unique values from each other and the other variables listed above so that different commands can be differentiated. `SCAN_RATE` allows the scan rate to be adjusted in the sketch from Python, `V_MIN` and `V_MAX` allow for the minimum and maximum voltages of the triangular wave to be adjusted, and `N_CYCLES` allows for the number of cycles of the triangular wave to be adjusted.
- `readPin1` and `readPin2`: These are analog pins from which the triangular wave is read after its range is "squished" and the signal from the working electrode after being inverted and shifted are read respectively.
- `diffPin`: The PWM pin that is used to shift down the triangular wave in the difference amplifier.
- `scanRate`: I initialized the scan rate variable at 100 mV/ms. This variable can be changed by Python and is not constant.
- `n_cycles`: I initialized the number of cycles that the triangle wave completes at 10 cycles. This value can change within the Bokeh app.
- `cycle_count`: Variable used to keep track of how many cycles of triangle waves have been completed. Initialized at 0
- scanRateStr, Vdiff_minStr, Vdiff_maxStr, n_cyclesStr: Strings intialized for reading in new values of variables from Python.
- `x`: Copied from lesson 16. This is the voltage value that will be read out to the MCP4725 to produce the trianglular wave.
- sampleDelay, lastSampleTime, startTime: These were copied from lesson 16. sampleDelay simply the delay between data samples collected (I chose 10 ms, as it was suggested to us to use as small of an interval as possible without overloading the data sent between Arduino and out computer. lastSampleTime is the last time a sample was collected and startTime is the time that the time the sketch started running.
- daqMode, daqDelay, and daqDelayString: Copied from lesson 16. These are to initialize the sketch as not running until requested from Python, delay data acquisition, and store the last data acquisition.
- `V_min` and `V_max`: Initializing the initial minimum and maximum voltage of the triangular wave after being "squished" and shifted down by the difference amplifier.



### Initial Math for Triangular Wave:
Since we are trying to produce a triangular wave that changes based on various parameters including: `scanRate`, `V_min`, and `V_max`, this involves creating functions that can produce different triangular waves based on these parameters. While I did not directly create a triangle wave until the `triangleWave()` function, I did set up some math for the initial values required to create the wave when initializing the variables. Calculations for the triangle wave were made following advice from Heather given during office hours to generate the triangular wave as a function of time and define the slope using: $\delta t = \frac{\delta E_{step}}{v}$

First, I computed `V_diff`, the difference between the minimum and maximum voltage in units of voltage. In my circuit design, the maximum voltage difference is 3V instead of 5V, because in my difference amplifier, I scaled the triangular wave by a factor of 3/5 to maximize the range of the wave. Since the full range of the DAQ is 4095, this means that for a voltage difference of 3 V, the full 4095 range of the DAQ is being used. However, when a smaller range is selected in my Bokeh app, my wave needs to scale to be smaller as well to change with the selected range. Due to the scaling of 3/5, this means that a 1V range is now 4095/3 instead of 4095/5. Thus, to convert a voltage into one that can be read by the DAQ and scale it correctly, I need to multiply the voltage by 4095/3 as shown below. I selected these units, as I would need to change the slope of the function to be negative once it reached `V_diff` or the maximum voltage of the entirely positive, unscaled triangle wave.


$V_{diff} = (V_{max}-V_{min}) * \frac{4095}{3}$

Next, I need to compute how much I want the voltage to increase by for each change in the triangle wave. Since there are only finite points on the triangle wave (a maximum of 4095 up and 4095 down), it ends up being essentially a staircase up and down. This is because we increase the voltage a constant amount at a constant interval based on different parameters. We want these steps to be as small as possible to create a wave as close to true triangle wave (infinite points) as possible. I simply divided V_diff by 4095, so that the steps were scaled by 1 for the maximum range, making the steps quite small. I stored this variable as a float, so that it wouldn't equal 0 for values below 1.

$E_{step} = \frac{V_{diff}}{4095}$

**Look over if have time**

Finally, I could compute the slope of the triangle wave function. To do this, I first had to ensure that `V_diff` and `scanRate` were in the same units. `V_diff` was in units of the DAQ signal, while `scanRate` was in mV/s. Thus, I chose to convert the `scanRate` into units of the DAQ signal as shown below. Again, the DAQ units are adjusted by a scale of 3, since that's the full range of the wave after being squished.

$\frac{v mV}{1 s} * \frac{1s}{1000ms} * \frac{1V}{1000mV} * \frac{4095 DAQ units}{3 V} = v * 1.365 * 10^{-3} DAQ units/s$, where $v$ is the scan rate.

Thus, the slope of the function can be described by:

$V_{slope} = \frac{E_{step}}{v*1.365*10^{-3}}$


The last value that needs to be calculated is the shift down of the triangle wave. This is the duty ratio that will be used in the PWM signal to shift down the wave in the difference amplifier.

V_diff_amp = 255* V_max-V_min/2 -255*(V_min+V_max)/ 2


**Figure out**

### change_V_range():
In the `change_V_range()` function, I created a function to update the `V_diff`, `E_step`, `V_slope`, and `V_diff_amp` when they are changed within Python. This function is called in the different cases in the `loop()` function, which I will detail later. The values are calculated the same way as in the *Initial Math for Triangular Wave* section. It works by updating the global variables from that section. Additionally, I reset the variable `cycle_count` to 0, since we would want to reach the total number of cycles with the updated wave.


### triangleWave():
In the `triangleWave()` function, it computes a value, `x` that is sent out to the MCP2725 based on the time, slope of the function, and the current value of the function. It takes in the unsigned long, `currTime`, which is the current time computed in the `loop()`. 

I created a variable, `time_val`, which computes the difference between `currTime` and `startTime` or the current time minus the start time. Then, it multiplies this value by `V_slope`, the increase in voltage of the triangle wave per ms as computed in the *Initial Math for Triangular Wave* or `change_V_range()`. This gives us a voltage value based on the time. I then modded this value by `V_diff` * 2. Since `V_diff` is the maximum voltage the wave can be, it is the inflection point where the slope goes from being positive to negative. When `V_diff` is reached again, the triangle wave is at 0V, another inflection point where the slope goes from negative to positive. Thus, a full triangle wave is completed at `V_diff` * 2. Therefore, a full wave cycle can be completed by modding the slope times time by `V_diff` * 2.

The first `if` statement is for when the slope is positive or rather when `time_val < V_diff` + 1. Here, we can directly set `x` equal to `time_val` since it is a positive integer less than or equal to 4095. The second `if` statement is simply to keep track of the number of wave cycles have passed. When `time_val == V_diff`, this means we have reached a peak of the triangular wave. Thus, we can add 1 to the number of wave cycles that have passed. Lastly, we have our `else` statement, where our slope is negative. Instead of computing a new value for `x`, we can simply take `V_diff * 2 - time_val`, since the full cycle is V_diff * 2 and time_value will increase over time, decreasing the number equivalently. Finally, I used `dac.setVoltage(x, false)` to set the voltage of the MCP4725 to x.


### printVoltage():
I modified the `printVoltage()` function from lesson 16 to read in voltages from `readPin1` and `readPin2`, reading in the shifted triangular wave and signal from the working electrode respectively. Then, I collected the time and used an `if` statement to check if serial was available to write. If it was, I printed the time in milliseconds, voltage of the shifted triangular wave, and the voltage of the working electrode.


### Setup():
In my `setup()` function, I initialized serial communication, using `Serial.begin()` and a baud rate of 15200. I also set up communication with the DAC using `dac.begin(MCP4725_ADDR)` as defined earlier. I also set my analog pins as outputs and my PWM pin as an input using `pinMode()`, initialized my startTime, and wrote out my initial PWM voltage to the difference amplifier.


### loop():
I used the `loop()` function from lesson 16 as a guideline for my sketch. First, I checked to see if my mode was to stream data. If it was, I ensured that the number of cycles was less than or equal to the maximum number of cycles. If it was, then, I updated `currTime` with the current time and called `triangleWave()`, as it was suggested to us to have a steady triangle wave being sent out regardless of the time when streaming is on. Then, if enough time had elapsed since the last sample collection, I called `printVoltage()` to print out the data to Serial.

In the next section, I had all of the different cases for things that could be controlled in Python. First, it checked to see if information had been sent to Arduino from Python. If so, it read in the information from Python. If data was requested, then the different cases could occur. In the case of a voltage request, `printVoltage()` was called. In the case of on request, the daqMode was switched to on request, while if the case was stream, daqMode was switched to stream. In the case of HANDSHAKE, it would check if the serial connection was ready to communicate with Arduino and print a message.

As for the cases I created, in the case, `SCAN_RATE`, `scanRate` was changed and `change_V_range()` was called to update the values for the triangle wave. In the cases, `V_MIN` and `V_MAX`, the `V_min` and `V_max` values were changed and `change_V_range()` was called. Finally, in the case `N_CYCLES`, the variable `n_cycles` was changed to match the new number of cycles allowed and the old `cycle_count` was reset to 0.

## **Sketch**

```cpp


// Adafruit provides a convenient library!
#include <Adafruit_MCP4725.h>

//This is the I2C Address of the MCP4725, by default (A0 pulled to GND).
//For devices with A0 pulled HIGH, use 0x63
#define MCP4725_ADDR 0x62


// Instantiate the convenient class
Adafruit_MCP4725 dac;

const int HANDSHAKE = 0;
const int VOLTAGE_REQUEST = 1;
const int ON_REQUEST = 2;
const int STREAM = 3;
const int READ_DAQ_DELAY = 4;
const int SCAN_RATE = 5;
const int V_MIN = 6;
const int V_MAX = 7;
const int N_CYCLES = 8;


const int readPin1 = A0;
const int readPin2 = A1;
const int diffPin = 3;
int scanRate = 100;
int n_cycles = 10;
int cycle_count = 0;

String scanRateStr, Vdiff_minStr, Vdiff_maxStr, n_cyclesStr;

uint16_t x = (uint16_t) 0;

const unsigned long sampleDelay = 10;
unsigned long lastSampleTime = 0;
unsigned long startTime = 0;

// Initially, only send data upon request
int daqMode = ON_REQUEST;

// Default time between data acquisition is 100 ms
int daqDelay = 100;

// String to store input of DAQ delay
String daqDelayStr;


// Keep track of last data acquistion for delays
unsigned long timeOfLastDAQ = 0;

// Max and min voltage after differential amplifier
float V_min = -1.5;
float V_max = 1.5;

// Calculating V_slope for triangle wave
int V_diff = (V_max-V_min) * 4095/3.0;
float E_step = V_diff / 4095.0;
float V_slope = E_step / (scanRate/1000.0);

// Calculating shift down for differential amplifier
int V_diff_amp = -255*(V_min+V_max)/ 2;


void change_V_range(float V_min, float V_max){
  
  V_diff = (V_max-V_min) * 4095/3.0;
  E_step = V_diff / 4095.0;
  V_slope = E_step / (scanRate/1000.0);
  
  V_diff_amp = -255*(V_min+V_max)/ 2;

  cycle_count = 0;
  
  analogWrite(diffPin, V_diff_amp);
  
}

void triangleWave(unsigned long currTime){
  int time_val = int(V_slope *(currTime-startTime)) % (V_diff*2);

   if (time_val < V_diff+1){
     x = (uint16_t)(time_val);
     if(time_val==V_diff){
      cycle_count ++;
     }
    
   }
   
   else{
    x = (uint16_t)(int((V_diff*2)-time_val));
   }
  
  dac.setVoltage(x, false);
}

unsigned long printVoltage() {
  // Read value from analog pin
  int triangle = analogRead(readPin1);
  int cyclic_volt = analogRead(readPin2);

  // Get the time point
  unsigned long timeMilliseconds = millis();

  // Write the result
  if (Serial.availableForWrite()) {
    String outstr = String(String(timeMilliseconds, DEC) + "," + String(triangle, DEC) + "," + String(cyclic_volt, DEC));
    Serial.println(outstr);
  }

  // Return time of acquisition
  return timeMilliseconds;
}


void setup() {
  // Initialize serial communication
  Serial.begin(115200);
  dac.begin(MCP4725_ADDR);

  pinMode(readPin1, INPUT);
  pinMode(readPin2, INPUT);
  startTime = millis();

  pinMode(diffPin, OUTPUT);

  analogWrite(diffPin, V_diff_amp);
}


void loop() {
  // If we're streaming
  if (daqMode == STREAM) {
    if(cycle_count < n_cycles + 1){
      unsigned long currTime = millis();
      triangleWave(currTime);
      if (millis() - timeOfLastDAQ >= daqDelay) {
        timeOfLastDAQ = printVoltage();
      }
    }
  }

  // Check if data has been sent to Arduino and respond accordingly
  if (Serial.available() > 0) {
    // Read in request
    int inByte = Serial.read();
  
    // If data is requested, fetch it and write it, or handshake
    switch(inByte) {
      case VOLTAGE_REQUEST:
        timeOfLastDAQ = printVoltage();
        break;
      case ON_REQUEST:
        daqMode = ON_REQUEST;
        break;
      case STREAM:
        daqMode = STREAM;
        break;
      case READ_DAQ_DELAY:
        // Read in delay, knowing it is appended with an x
        daqDelayStr = Serial.readStringUntil('x');
  
        // Convert to int and store
        daqDelay = daqDelayStr.toInt();
  
        break;
      case HANDSHAKE:
        if (Serial.availableForWrite()) {
          Serial.println("Message received.");
        }
        break;
      case SCAN_RATE:
        scanRateStr = Serial.readStringUntil('x');
        scanRate = scanRateStr.toInt();
        change_V_range(V_min, V_max);
        break;
      case V_MIN:
        Vdiff_minStr = Serial.readStringUntil('x');
        V_min = Vdiff_minStr.toInt();
        change_V_range(V_min, V_max);
        break;
      case V_MAX:
        Vdiff_maxStr = Serial.readStringUntil('x');
        V_max = Vdiff_maxStr.toInt();
        change_V_range(V_min, V_max);
        break;
      case N_CYCLES:
        n_cyclesStr = Serial.readStringUntil('x');
        n_cycles = n_cyclesStr.toInt();
        cycle_count = 0;
        break;
      }
    }
}
```

## **Python Code:**
Much of the Python code is adapted from lesson 16. Sections that were directly copied are labeled as such. The majority of coding I did was to add additional widgets to the Bokeh app that could update values in Arduino, read in 3 values from Arduino, and math for computing concentrations.

### Streaming Data from Arduino:
Since I wanted plots that streamed live data from Arduino, I needed to stream data through the Serial connection between Python and Jupyter. The functions for finding Arduino and opening the serial connection were copied directly from lesson 16. However, I did update many of the streaming functions to read in 3 values from Arduino instead of 2, as I needed to read in the time, voltage of the triangle wave, and the output of the working electrode.

### Streaming Plots:
Since I wanted to stream both plots for the triangle wave, working electrode output, and the duck plot, this meant that I needed to stream 2 plots. To stream the triangle wave and working electrode output, I kept the original plot function and updated it to stream 2 lines with respect to time. This simply required me to add an additional line to the plot within the `plot()` function.I also added a legend to differentiate the two lines. The triangle wave that is being plotted has been squished by the difference amplifier, while the working electrode output being plotted is after the summing amplifier.  

To create the duck plot, I defined an additional plotting function, `duck_plot()` to plot the current of the working electrode against the input voltage from the triangle wave, which is why it couldn't be plotted with the other curves. I streamed in the data the same way as the `plot()` function.

I also removed the request voltage plot and functions and variables regarding this from lesson 16.

### Creating Widgets:
I wanted to create sliders for the scan rate, minimum and maximum voltages, and the number of cycles of the wave. I chose sliders, as they're easy to use, allow me to set reasonable ranges that work with the code, and are a good visual representation of how changing the varaibles impacts the triangle wave and the duck plot. To create the scan rate and number of cycle sliders, I used the `bokeh.models.slider()` widget. 

For the scan rate, I chose a minimum scan rate of 50 mV/s and a maximum scan rate of 300 mV/s, as that these were the minimum and maximum scan rates of the vitamin C paper. I chose a step of 25, as this gave 12 total scan rates, which seemed like enough to collect good data while not having awkward intervals and set the default value to 100 mV/s, as it was a value somewhat in the middle and is a nice number. 

For the number of cycles, I set the range between 1 and 30, as if someone only wanted 1 wave to complete, they could look at it closely, while if they wanted a larger dataset, 30 waves would give plenty of data. I didn't want it to go much higher since it would take quite awhile for it to complete. I set the default value to 10 since this seemed like a decent number of cycles to complete and set the step to 5, as this was enough to see a difference but still have decent control of the number of waves.

For the voltage range slider, I used the `bokeh.models.widgets.RangeSlider()` to produce a slider with a range of voltages. Since in my circuit, the maximum range is 3V, it cannot go below this

**Fix rangeSlider**

I kept the widgets from lesson 16 and adjusted them to work for 2 voltages instead of 1. This included the stream, reset, save, and shut down buttons.

### Widget Callbacks:
To ensure that the widgets were actually changing values in Arduino, I had to write callback functions and tie them to the Arduino widgets. I wrote 3 functions to accomplish this, `scan_rate_callback()`, `V_range_callback()`, and `N_cycles_callback()`. These functions read in the information about connecting to Arduino and new, the new value from the widgets. Then, they use `arduino.write()` to write a message to Arduino, which includes which case in the `loop()` function is occuring and the value of the variable that needs to change.


Calls for these functions were added to the `potentiometer_app()` function and connected to the widgets using the `.on_change()` method, so that when the widgets are altered, the callback functions would be called and alter the values of variables in Arduino.

### Calculating Concentrations:
I wrote 3 functions to calculate the concentration. The first function determines the anodal peaks of the duck plot. The second function produces a linear regression with respect to the scan rate to determine D. The third function produces a linear regression to determine the concentration of the solution.

# **Section 3: Instructions for Use:**

### **Buttons:**
- Stream button: Green button below the plots to the left. Pressing this button once will stream data in from the Arduino. Pressing it again will terminate streaming.
- Reset button: Yellow button below the plots in the center. Pressing this button will reset all of the streamed data.
- Save button: This is the blue button to the right of the plots. It is made up of a textbox, where you can title the .csv file you would like to save, and the save button which will save all of the streamed data to a csv with the given name.
- Shut down button: Red button to the bottom right of the app. Pressing this button will restart the kernal, resetting all variables. All cells of the notebook will need to be run again to use the app.

### **Sliders:**
- Scan rate: This varies the scan rate of the triangle wave. The slider is in mV/s. This alters the speed at which the voltage of the triangle wave changes.
- Voltage range: This is the voltage range or amplitude of the triangular wave. You might not see the full change of the triangle wave on the plot, because Arduino cannot read in negative voltages, so it will be cut off.
- Number of cycles: This is the number of full triangle waves that are completed. The plot will stop streaming after this number of full triangle waves have been sent.

### **General Use:**
1. Ensure Arduino is plugged into computer and the powerBRICK is connected to power.
2. Upload the Arduino file to the Arduino and run all cells of this Jupyter notebook.
3. Place 1-2 drops of solution onto the electrode (ensuring that you cover the black dot in the center and the silver surrounding it)
4. Press the Stream button (the green button below and to the left of the plots).
5. Change the values of the sliders as desired (more notes in next section).
6. Wait until plot stops streaming (this means the selected number of cycles has completed)
7. Press the yellow reset button (below the plots) if you would like to repeat.

### **Collecting Data for Determining Concentration:**
1. Ensure Arduino is plugged into computer and the powerBRICK is connected to power.
2. Upload the Arduino file to the Arduino and run all cells of this Jupyter notebook.
3. Place 1-2 drops of solution onto the electrode (ensuring that you cover the black dot in the center and the silver surrounding it)
4. Use the sliders to determine the parameters you would like to collect your data set with (the scan rate, voltage range, and number of wave cycles). The sliders are below the plots. You will want to vary to scan rate to among different datasets to determine concentration.
5. Press the Stream button (the green button below and to the left of the plots).
6. Save your data by naming your dataset and pressing the blue save button (to the right of the plots). It is reccommended that you title the data something of the form *sample_scanRate_voltageRange_numberCycles.csv*
7. Press the yellow reset button (below the plots) and repeat this process for different parameters
8. After all of the data has been collected, call the **ADD FUNCTION HERE** to compute the concentration.

# **Python Code**
### Setting up Serial connection with Arduino

Functions copied from lesson 16. These functions were not altered

In [2]:
def find_arduino(port=None):
    """Get the name of the port that is connected to Arduino."""
    if port is None:
        ports = serial.tools.list_ports.comports()
        for p in ports:
            if p.manufacturer is not None and "Arduino" in p.manufacturer:
                port = p.device
    return port


def handshake_arduino(
    arduino, sleep_time=1, print_handshake_message=False, handshake_code=0
):
    """Make sure connection is established by sending
    and receiving bytes."""
    # Close and reopen
    arduino.close()
    arduino.open()

    # Chill out while everything gets set
    time.sleep(sleep_time)

    # Set a long timeout to complete handshake
    timeout = arduino.timeout
    arduino.timeout = 2

    # Read and discard everything that may be in the input buffer
    _ = arduino.read_all()

    # Send request to Arduino
    arduino.write(bytes([handshake_code]))

    # Read in what Arduino sent
    handshake_message = arduino.read_until()

    # Send and receive request again
    arduino.write(bytes([handshake_code]))
    handshake_message = arduino.read_until()

    # Print the handshake message, if desired
    if print_handshake_message:
        print("Handshake message: " + handshake_message.decode())

    # Reset the timeout
    arduino.timeout = timeout


def read_all(ser, read_buffer=b"", **args):
    """Read all available bytes from the serial port
    and append to the read buffer.

    Parameters
    ----------
    ser : serial.Serial() instance
        The device we are reading from.
    read_buffer : bytes, default b''
        Previous read buffer that is appended to.

    Returns
    -------
    output : bytes
        Bytes object that contains read_buffer + read.

    Notes
    -----
    .. `**args` appears, but is never used. This is for
       compatibility with `read_all_newlines()` as a
       drop-in replacement for this function.
    """
    # Set timeout to None to make sure we read all bytes
    previous_timeout = ser.timeout
    ser.timeout = None

    in_waiting = ser.in_waiting
    read = ser.read(size=in_waiting)

    # Reset to previous timeout
    ser.timeout = previous_timeout

    return read_buffer + read


def read_all_newlines(ser, read_buffer=b"", n_reads=4):
    """Read data in until encountering newlines.

    Parameters
    ----------
    ser : serial.Serial() instance
        The device we are reading from.
    n_reads : int
        The number of reads up to newlines
    read_buffer : bytes, default b''
        Previous read buffer that is appended to.

    Returns
    -------
    output : bytes
        Bytes object that contains read_buffer + read.

    Notes
    -----
    .. This is a drop-in replacement for read_all().
    """
    raw = read_buffer
    for _ in range(n_reads):
        raw += ser.read_until()

    return raw

## Creating Arduino Callbacks

In [3]:
def scan_rate_callback(arduino, new):
    arduino.write(bytes([SCAN_RATE]) + (str(new) + "x").encode())

def V_range_callback(arduino, new):
    Vmin, Vmax = new
    arduino.write(bytes([V_MIN])+ (str(Vmin) + "x").encode())
    arduino.write(bytes([V_MAX])+ (str(Vmax) + "x").encode())

def N_cycles_callback(arduino, new):
    arduino.write(bytes([N_CYCLES])+ (str(new) + "x").encode())

## Functions for requesting and reading in voltages

In [4]:
def parse_read(read):
    """Parse a read with time, volage data

    Parameters
    ----------
    read : byte string
        Byte string with comma delimited time/voltage
        measurements.

    Returns
    -------
    time_ms : list of ints
        Time points in milliseconds.
    voltage : list of floats
        Voltages in volts.
    remaining_bytes : byte string
        Remaining, unparsed bytes.
    """
    time_ms = []
    V_tri_lst = []
    V_cyc_lst = []

    # Separate independent time/voltage measurements
    pattern = re.compile(b"\d+|,")
    raw_list = [
        b"".join(pattern.findall(raw)).decode()
        for raw in read.split(b"\r\n")
    ]

    for raw in raw_list[:-1]:
        try:
            t, V_tri, V_cyc = raw.split(",")
            time_ms.append(int(t))
            V_tri_lst.append(int(V_tri) * 5 / 1023)
            V_cyc_lst.append(int(V_cyc) * 5 / 1023)
        except:
            pass

    if len(raw_list) == 0:
        return time_ms, V_tri_lst, V_cyc_lst, b""
    else:
        return time_ms, V_tri_lst, V_cyc_lst, raw_list[-1].encode()


def parse_raw(raw):
    """Parse bytes output from Arduino."""
    raw = raw.decode()
    if raw[-1] != "\n":
        raise ValueError(
            "Input must end with newline, otherwise message is incomplete."
        )

    t, V_tri, V_cyc = raw.rstrip().split(",")
    V_tri = int(V_tri) * 5 / 1023
    V_cyc = int(V_cyc) * 5 / 1023
    

    return int(t), V_tri, V_cyc


def request_single_voltage(arduino):
    """Ask Arduino for a single data point"""
    # Ask Arduino for data
    arduino.write(bytes([VOLTAGE_REQUEST]))

    # Read in the data
    raw = arduino.read_until()

    # Parse and return
    return parse_raw(raw)

In [5]:
# Set up connection
HANDSHAKE = 0
VOLTAGE_REQUEST = 1
ON_REQUEST = 2;
STREAM = 3;
READ_DAQ_DELAY = 4;
SCAN_RATE = 5;
V_MIN = 6;
V_MAX = 7;
N_CYCLES = 8;


port = find_arduino()
arduino = serial.Serial(port, baudrate=115200)
handshake_arduino(arduino)

In [6]:
# Set up data dictionaries
stream_data = dict(prev_array_length=0, t=[], V_tri=[], V_cyc=[], mode="on demand")

In [7]:
async def daq_stream_async(
    arduino, data, delay=10, n_trash_reads=5, n_reads_per_chunk=4, reader=read_all_newlines
):
    """Obtain streaming data"""
    # Specify delay
    arduino.write(bytes([READ_DAQ_DELAY]) + (str(delay) + "x").encode())

    # Current streaming state
    stream_on = False

    # Receive data
    read_buffer = [b""]
    while True:
        if data["mode"] == "stream":
            # Turn on the stream if need be
            if not stream_on:
                arduino.write(bytes([STREAM]))

                # Read and throw out first few reads
                i = 0
                while i < n_trash_reads:
                    _ = arduino.read_until()
                    i += 1

                stream_on = True

            # Read in chunk of data
            raw = reader(
                arduino, read_buffer=read_buffer[0], n_reads=n_reads_per_chunk
            )

            # Parse it, passing if it is gibberish
            try:
                t, V_tri, V_cyc, read_buffer[0] = parse_read(raw)

                # Update data dictionary
                data["t"] += t
                data["V_tri"] += V_tri
                data["V_cyc"] += V_cyc
            except:
                pass
        else:
            # Make sure stream is off
            stream_on = False

        # Sleep 80% of the time before we need to start reading chunks
        await asyncio.sleep(0.8 * n_reads_per_chunk * delay / 1000)


daq_task = asyncio.create_task(daq_stream_async(arduino, stream_data))

## Functions for Plotting

In [8]:
def plot(mode):
    """Build a plot of voltage vs time data"""
    # Set up plot area
    p = bokeh.plotting.figure(
        frame_width=500,
        frame_height=175,
        x_axis_label="time (s)",
        y_axis_label="voltage (V)",
        title="Triangle Wave and Incoming Signal",
        y_range=[-0.2, 5.2],
        toolbar_location="above",
    )

    # No range padding on x: signal spans whole plot
    p.x_range.range_padding = 0

    # We'll sue whitesmoke backgrounds
    p.border_fill_color = "whitesmoke"

    # Defined the data source
    source = bokeh.models.ColumnDataSource(data=dict(t=[], V_tri=[], V_cyc=[]))

    # If we are in streaming mode, use a line, dots for on-demand
    if mode == 'stream':
        p.line(source=source, x="t", y="V_tri", legend_label="triangle wave")
        p.line(source=source, x="t", y="V_cyc", color="red", legend_label="cyclic voltammeter")
    else:
        p.circle(source=source, x="t", y="V_tri")
        p.circle(source=source, x="t", y="V_cyc", color="red")
        
    p.legend.location = "top_right"
    p.legend.click_policy = "hide"

    # Put a phantom circle so axis labels show before data arrive
    phantom_source = bokeh.models.ColumnDataSource(data=dict(t=[0], V_tri=[0], V_cyc=[0]))
    p.circle(source=phantom_source, x="t", y="V_tri", visible=False)

    return p, source, phantom_source

In [9]:
def duck_plot(mode, source, phantom_source):
    """Build a plot of voltage vs time data"""
    # Set up plot area
    duck_plot = bokeh.plotting.figure(
        frame_width=500,
        frame_height=175,
        x_axis_label="Applied Potential (V)",
        y_axis_label="Current (uA)",
        title="Duck Plot",
        toolbar_location="above",
    )


    # We'll sue whitesmoke backgrounds
    duck_plot.border_fill_color = "whitesmoke"

    # Defined the data source
    duck_source = bokeh.models.ColumnDataSource(data=dict(t=[], V_tri=[], V_cyc=[]))

    # If we are in streaming mode, use a line, dots for on-demand
    if mode == 'stream':
        duck_plot.line(source=source, x="V_cyc", y="V_tri")
    else:
        duck_plot.circle(source=source, x="V_cyc", y="V_tri")
        


    return duck_plot

In [10]:
def controls(mode):
    if mode == "stream":
        acquire = bokeh.models.Toggle(label="stream", button_type="success", width=100)
        save_notice = bokeh.models.Div(
            text="<p>No streaming data saved.</p>", width=165
        )
    else:
        acquire = bokeh.models.Button(label="acquire", button_type="success", width=100)
        save_notice = bokeh.models.Div(
            text="<p>No on-demand data saved.</p>", width=165
        )
    
    
    save = bokeh.models.Button(label="save", button_type="primary", width=100)
    reset = bokeh.models.Button(label="reset", button_type="warning", width=100)
    file_input = bokeh.models.TextInput(
        title="file name", value=f"{mode}.csv", width=165
    )
    
    scan_rate_slider = bokeh.models.Slider(start=50, end=300, value=100, step=25, title="scan rate")
    V_range_slider = bokeh.models.widgets.RangeSlider(title="voltage range", 
                                                      start=-2.5, 
                                                      end=2.5, 
                                                      value=(-1.5, 1.5), 
                                                      step=0.1,
                                                     )
    n_cycles_slider = bokeh.models.Slider(start=1, end=30, value=10, step=5, title="number of cycles")

    return dict(
        acquire=acquire,
        reset=reset,
        save=save,
        file_input=file_input,
        save_notice=save_notice,
        scan_rate_slider=scan_rate_slider,
        V_range_slider=V_range_slider,
        n_cycles_slider=n_cycles_slider,
        
    )

In [11]:
def layout(p_stream, duck_plot, ctrls):
    buttons = bokeh.layouts.row(
        bokeh.models.Spacer(width=30),
        ctrls["acquire"],
        bokeh.models.Spacer(width=295),
        ctrls["reset"],
    )
    left = bokeh.layouts.column(p_stream,
                                duck_plot,
                                buttons)
    right = bokeh.layouts.column(
        ctrls["file_input"],
        ctrls["save"],
        ctrls["save_notice"],
    )
    
    top = bokeh.layouts.row(
        left, right, spacing=30, margin=(30, 30, 30, 30), background="whitesmoke",
    )
    
    
    
    return bokeh.layouts.column(
        top,
        ctrls["scan_rate_slider"],
        ctrls["V_range_slider"],
        ctrls["n_cycles_slider"],
        spacing=30, margin=(30, 30, 30, 30), background="whitesmoke",
    )

### Code for Widgets

In [12]:
def acquire_callback(arduino, rollover):
    # Pull t and V values from stream or request from Arduino
    if stream_data["mode"] == "stream":
        t = stream_data["t"][-1]
        V_tri = stream_data["V_tri"][-1]
        V_cyc = stream_data["V_cyc"][-1]
        
            
        
    else:
        t, V_tri, V_cyc = request_single_voltage(arduino)


    # Send new data to plot
    new_data = dict(t=[t / 1000], V_tri=[V_tri], V_cyc=[V_cyc])
    source.stream(new_data, rollover=rollover)

    # Update the phantom source to keep the x_range on plot ok
    phantom_source.data = new_data

In [13]:
def stream_callback(arduino, stream_data, new):
    if new:
        stream_data["mode"] = "stream"
    else:
        stream_data["mode"] = "on-demand"
        arduino.write(bytes([ON_REQUEST]))

    arduino.reset_input_buffer()

In [14]:
def reset_callback(mode, data, source, phantom_source, controls):
    # Turn off the stream
    if mode == "stream":
        controls["acquire"].active = False

    # Black out the data dictionaries
    data["t"] = []
    data["V_tri"] = []
    data["V_cyc"] = []

    # Reset the sources
    source.data = dict(t=[], V_tri=[], V_cyc=[])
    phantom_source.data = dict(t=[0], V_tri=[0], V_cyc=[0])

In [15]:
def save_callback(mode, data, controls):
    # Convert data to data frame and save
    df = pd.DataFrame(data={"time (ms)": data["t"], 
                            "voltage triangle(V)": data["V_tri"],
                            "voltage cyclic volt(V)": data["V_cyc"],
                           })
    df.to_csv(controls["file_input"].value, index=False)

    # Update notice text
    notice_text = "<p>" + ("Streaming" if mode == "stream" else "On-demand")
    notice_text += f" data was last saved to {controls['file_input'].value}.</p>"
    controls["save_notice"].text = notice_text

In [16]:
def disable_controls(controls):
    """Disable all controls."""
    for key in controls:
        controls[key].disabled = True


def shutdown_callback(
    arduino, daq_task, stream_data, stream_controls
):
    # Disable controls
    disable_controls(stream_controls)

    # Strop streaming
    stream_data["mode"] = "on-demand"
    arduino.write(bytes([ON_REQUEST]))

    # Stop DAQ async task
    daq_task.cancel()

    # Disconnect from Arduino
    arduino.close()

### Updating Data as it Streams

In [17]:
def stream_update(data, source, phantom_source, controls, rollover):
    # Update plot by streaming in data
    new_data = {
        "t": np.array(data["t"][data["prev_array_length"] :]) / 1000,
        "V_tri": data["V_tri"][data["prev_array_length"] :],
        "V_cyc": data["V_cyc"][data["prev_array_length"] :],
    }
    source.stream(new_data, rollover)
    

    # Adjust new phantom data point if new data arrived
    if len(new_data["t"]) > 0:
        phantom_source.data = dict(t=[new_data["t"][-1]], 
                                   V_tri=[new_data["V_tri"][-1]],
                                   V_cyc=[new_data["V_cyc"][-1]],
                                  )
    data["prev_array_length"] = len(data["t"])

In [18]:
def potentiometer_app(
    arduino, stream_data, daq_task, rollover=400, stream_plot_delay=90,
):
    def _app(doc):
        # Plots
        p_stream, stream_source, stream_phantom_source = plot("stream")
        p_duck = duck_plot("stream", stream_source, stream_phantom_source)
        

        # Controls
        stream_controls = controls("stream")

        # Shut down
        shutdown_button = bokeh.models.Button(
            label="shut down", button_type="danger", width=100
        )

        # Layouts
        stream_layout = layout(p_stream, p_duck, stream_controls)

        # Shut down layout
        shutdown_layout = bokeh.layouts.row(
            bokeh.models.Spacer(width=675), shutdown_button
        )

        app_layout = bokeh.layouts.column(
            stream_layout, shutdown_layout
        )

        def _acquire_callback(event=None):
            acquire_callback(
                arduino,
                stream_data,
                rollover,
            )

        def _stream_callback(attr, old, new):
            stream_callback(arduino, stream_data, new)

        def _stream_reset_callback(event=None):
            reset_callback(
                "stream",
                stream_data,
                stream_source,
                stream_phantom_source,
                stream_controls,
            )


        def _stream_save_callback(event=None):
            save_callback("stream", stream_data, stream_controls)


        def _shutdown_callback(event=None):
            shutdown_callback(
                arduino, daq_task, stream_data, stream_controls
            )
            
        def _scan_rate_callback(attr, old, new):
            scan_rate_callback(arduino, new)
            
        def _V_range_callback(attr, old, new):
            V_range_callback(arduino, new)
        
        def _N_cycles_callback(attr, old, new):
            N_cycles_callback(arduino, new)

        @bokeh.driving.linear()
        def _stream_update(step):
            stream_update(stream_data, stream_source, stream_phantom_source, stream_controls, rollover)

            # Shut down server if Arduino disconnects (commented out in Jupyter notebook)
            if not arduino.is_open:
                sys.exit()

        # Link callbacks
        stream_controls["acquire"].on_change("active", _stream_callback)
        stream_controls["reset"].on_click(_stream_reset_callback)
        stream_controls["save"].on_click(_stream_save_callback)
        shutdown_button.on_click(_shutdown_callback)
        
        stream_controls["scan_rate_slider"].on_change("value", _scan_rate_callback)
        stream_controls["V_range_slider"].on_change("value", _V_range_callback)
        stream_controls["n_cycles_slider"].on_change("value", _N_cycles_callback)
        
        # Add the layout to the app
        doc.add_root(app_layout)

        # Add a periodic callback, monitor changes in stream data
        pc = doc.add_periodic_callback(_stream_update, stream_plot_delay)

    return _app

In [19]:
bokeh.io.show(
    potentiometer_app(arduino, stream_data, daq_task),
    notebook_url=notebook_url,
)

# Calculating Concentration:

In [51]:
def find_ipa(data):
    ipa_peaks = sp.signal.find_peaks(data["V_cyc"])[0]
    ipa = np.take(data["V_cyc"], ipa_peaks)
    scan_rate = np.take(data["scan_rate"], ipa_peaks)
    return (ipa, scan_rate)

def find_D(data, C, T=298, n=2):
    # Defining Constants
    k_B = 1.38064852 * 10**(-23)
    ne = 1.60217662 * 10**-19 * n
    
    # Figure out actual value later
    A = 4 * 10**(-3)
    
    
    ipa, scan_rate = find_ipa(data)
    scan_rate = np.sqrt(scan_rate)
    slope, intercept, r, p, se = sp.stats.linregress(scan_rate, ipa)
    D = k_B*T/ne * (slope / (0.4463*ne*A))**2
    return D

def find_C(data, D, T=298, n=2):
    # Defining Constants
    k_B = 1.38064852 * 10**(-23)
    ne = 1.60217662 * 10**-19 * n
    
    # Figure out actual value later
    A = 4 * 10**(-3)
    
    
    ipa, scan_rate = find_ipa(data)
    scan_rate = np.sqrt(scan_rate)
    slope, intercept, r, p, se = sp.stats.linregress(scan_rate, ipa)
    c = slope / (0.4463*ne*A) * math.sqrt(k_B*T / ne*D)
    return c
    

In [22]:
# All at scan rate of 500
df_0 = pd.read_csv('0_stream.csv')
df_6_25 = pd.read_csv('0.625_stream.csv')
df_10 = pd.read_csv('10mm_stream.csv')
df_2_5 = pd.read_csv('2.5mm_stream.csv')
df_5 = pd.read_csv('5mm_stream.csv')



# **Section 4: Demonstration and Assessment:**

The main criteria I wanted to test my cyclic voltammeter on was whether I could generate a triangle wave that changed with all of my widgets and was accurate to the parameters, if I could properly shift and shrink the wave, if I could get readings out of the electrode and convert them into a voltage signal, if I could produce a duck plot, if I could successfully calculate D, and if I could successfully calculate the concentration of the mystery solution.

Unfortunately, I ran out of time and was not able to fully test out and fix some issues in my code. As a result, while I believe my circuit is fully functioning, I have some issues in my code that limit the capabilities of my cyclic voltammeter. Starting off with the circuit, I was able to successfully shrink down my triangle wave and shift it depending on my voltage range successfully. I was able to pass a clean wave signal into the electrode, which was quite successful. I was also able to get a signal out of my electrode and successfully converted it into a relatively clean voltage reading.

I was able to generate successful triangle waves as well. However, I realized very recently that my calculations for my waves were off and that they were not changing accurately when I changed the parameters in my code. Thus, I spent the last couple of days trying to resolve this issue. I was unfortunately quite busy with finals and had a bad migraine for a few days, meaning I couldn't spend as much time to fix it as I would have liked. I ended up with a triangle wave that changes with my widgets in the Bokeh app, I believe somewhat accurately. However, the triangle waves only appear to work about half of the time and the other half of the time, it seems to only plot half of the triangle wave. I spent hours trying to resolve this issue but was unsuccessful. 

I was still able to generate duck plots from both the less accurate triangle waves and during the periods of time when the triangle waves worked as expected. From these plots, I produced csv files for different concentrations of known solutions but ran out of time before I could complete the calculations for D and for the concentration of the mystery solution. I have saved these CSVs in a folder labeled *data_for_regress*. If I had been able to calculate these values, I would have compared the values I obtained to the actual value of D and the actual concentration of the mystery solution.

Ideally, I would've tested out the cyclic voltammeter on a solution that isn't vitamin C as well to determine if it can also be accurate with other solutions. It also would've been beneficial to attempt to predict the concentration of a known concentration using the same method as for the unknown species to determine how accurate it is. Then, I could've adjusted for any inaccuracies.

# **Section 5: Analysis of Data:**

As mentioned in part 4, I was unfortunately not fully able to fully complete my data analysis as I ran out of time. If I had enough time to complete my data analysis, I would have generated duck plots from all of the possible concentrations for different scan rates and created multiple regressions (one for each scan rate with the different concentrations). I would have calculated D from each of these regressions and used the mean as my value for D. Then, I would have used this value in the Randles-Sevcik equation and generated duck plots for the unknown concentration with respect to multiple scan rates and produced an additional linear regression to determine the concentration from this.

Again, I ideally would've also tested this on another set of concentrations to know that the cyclic voltammeter can accurately determine the concentration of other species.

The duck plots I generated seemed to generally follow the shape and structure of the duck plots on the website, with a clear anodal peak. There was still a bit of noise in the plots but they otherwise seemed to be decently accurate.

# **Section 6: Suggestions for Next Design Phase:**

In the next design phase, there are several modifications that could greatly enhance this cyclic voltammeter. Regarding the circuit itself, it would be beneficial to sauder the circuit and perhaps try to use a less sensitive electrode (if available). I've had several occasions where components on my breadboard suddenly stopped working due to something being a bit loosely connected. Saudering the components in place would prevent this issue. As for the electrode, I unfortunately had several of them corrode and I know that some of my classmates did as well. While I know part of the reason we used these electrodes was because they're small and only require a small volume of liquid, it might be beneficial to have slightly less sensitive electrodes, especially if the cyclic voltammeter would be under consistent use.

As for the digital aspects of the cyclic voltammeter, I was unfortunately not able to fully complete the analysis of concentration. While I wrote functions to in theory determine the diffusion coefficient, D and the concentration of the mystery solution, I ran out of time to actually implement them. Additionally, I had issues creating a fully functional triangle wave that both changed fully and correctly with all of the widgets and was a proper triangle wave. I spent many hours attempting to get a triangle wave that could be changed well with my widgets but was unsuccessful. I thought I had a functioning wave but I believe it's still having some issues. Thus, in the next design phase, it would be important for these pieces to be fully functioning so that it can actually calculate the concentration.

In [37]:
arduino.close()